In [433]:
import pandas as pd

import random
import nltk
#nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from num2words import num2words

# Reading Data


In [434]:
df = pd.read_csv("/home/mayank/Desktop/JEOPARDY_CSV.csv",names =["shownum","airdate","round","category","value","ques","ans"],header= None).dropna()

/home/mayank/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [435]:
df.columns = [col.strip() for col in df.columns]
df.shape

(216929, 7)

In [436]:
df = df.iloc[1:]
df.reset_index()
df = df.head(100000) # consider first 100k samples 


In [437]:
df = df.sample(frac=1).reset_index(drop=True) # reset index
df


,shownum,airdate,round,category,value,ques,ans
0,6182,2011-06-28,Jeopardy!,OUT OF JAIL,$600,"After about a year of incarceration, <a href=""...",(Lech) Walesa
1,5906,2010-04-26,Jeopardy!,1990s BESTSELLERS,$800,"In 1998 his novel ""Rainbow Six"" became another...",Tom Clancy
2,6113,2011-03-23,Double Jeopardy!,ISLANDS OF THE ATLANTIC,$1200,"Until 1995, the U.S. maintained an air base on...",Bermuda
3,2682,1996-04-09,Double Jeopardy!,INSECTS,$400,The firefly is not a true fly but one of these...,Beetle
4,4664,2004-12-09,Jeopardy!,SOCIAL TYPES,$800,One who wields his power unofficially is this ...,gray
...,...,...,...,...,...,...,...
99995,5306,2007-10-08,Jeopardy!,FAMOUS MOMS,$400,"In 2002 at age 101, the queen mother, this que...",Elizabeth II
99996,5744,2009-07-23,Jeopardy!,DANSWERS,$200,In February 1865 General William Tecumseh Sher...,Charleston
99997,3356,1999-03-22,Double Jeopardy!,COUNTRIES OF THE WORLD,$400,Muammar Al-Qaddafi is this country's head of s...,Libya
99998,4812,2005-07-05,Double Jeopardy!,BELIEFS,$1600,This eighth incarnation of Vishnu is often dep...,Krishna


In [438]:
# # Source: https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn
# class MultiColumnLabelEncoder:
#     def __init__(self,columns = None):
#         self.columns = columns # array of column names to encode

#     def fit(self,X,y=None):
#         return self # not relevant here

#     def transform(self,X):
#         '''
#         Transforms columns of X specified in self.columns using
#         LabelEncoder(). If no columns specified, transforms all
#         columns in X.
#         '''
#         output = X.copy()
#         if self.columns is not None:
#             for col in self.columns:
#                 output[col] = LabelEncoder().fit_transform(output[col])
#         else:
#             for colname,col in output.iteritems():
#                 output[colname] = LabelEncoder().fit_transform(col)
#         return output

#     def fit_transform(self,X,y=None):
#         return self.fit(X,y).transform(X)

In [439]:
# df = MultiColumnLabelEncoder(columns = ['value']).fit_transform(df)


In [440]:
# s = set()
# for i in range(len(df)):
#     s.add(df['category'][i])
# # Values have total 86 Different types of entires or dollors values 
# print(len(s))

# # There are total of 3429 different catagories


In [441]:
#Not considering all values:
# d = {"$200": 0,"$1000": 1}

# for i in range(len(df)):
# #     print(type(d[df["value"][i]]))
# #     break
#     if df['value'][i] in d:
#         df['value'][i] = d[df['value'][i]]
#     else:
#         df['value'][i] = 'None'
# df     




In [442]:
# df = df.sample(frac=1).reset_index(drop=True)
# df = df[df['value'] != 'None']


In [443]:
# Different prices in the dataset
from collections import Counter
print(Counter(df["value"]))

Counter({'$400': 19455, '$800': 14686, '$200': 14039, '$600': 9433, '$1000': 8972, '$1200': 5210, '$2000': 5189, '$1600': 4985, '$500': 4171, '$100': 4161, '$300': 4013, 'None': 1674, '$1,000': 965, '$2,000': 755, '$3,000': 353, '$1,500': 253, '$1,200': 207, '$4,000': 157, '$1,400': 112, '$1,600': 106, '$5,000': 101, '$700': 92, '$1,800': 89, '$2,500': 87, '$2,200': 67, '$2,400': 49, '$900': 42, '$2,600': 39, '$6,000': 36, '$1,300': 35, '$2,800': 28, '$3,600': 26, '$1,100': 26, '$3,200': 26, '$3,500': 26, '$3,400': 23, '$1,700': 21, '$4,200': 20, '$1,900': 18, '$7,000': 17, '$3,800': 14, '$4,800': 12, '$2,100': 12, '$10,000': 12, '$5,200': 11, '$2,300': 11, '$8,000': 8, '$4,500': 8, '$4,400': 8, '$4,600': 8, '$5': 7, '$6,600': 6, '$5,600': 6, '$5,400': 6, '$5,800': 6, '$6,400': 5, '$6,200': 5, '$9,000': 4, '$2,700': 4, '$3,300': 4, '$3,100': 4, '$7,200': 3, '$12,000': 3, '$3,700': 3, '$3,900': 3, '$10,800': 2, '$7,400': 2, '$750': 2, '$1,263': 2, '$4,100': 2, '$2,001': 2, '$7,800': 2, 

# Data Cleaning


In [444]:
import re

def clean_str(string):
    """Tokenization/string cleaning for all datasets except for SST.

    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"[.,#!$%&;:{}=_`~()/\\]", "", string)
    return string.strip().lower()



df['category'] = df['category'].astype(str).map(clean_str,na_action=None)
df['ques'] = df['ques'].astype(str).map(clean_str,na_action=None)
df['ans'] = df['ans'].astype(str).map(clean_str,na_action=None)


# Stemming and Lemmatization

from nltk.stem.porter import PorterStemmer

def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

df['category'] = get_stemmed_text(df['category'])
df['ques'] = get_stemmed_text(df['ques'])
df['ans'] = get_stemmed_text(df['ans'])

from nltk.stem import WordNetLemmatizer
def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

df['category'] = get_lemmatized_text(df['category'])
df['ques'] = get_lemmatized_text(df['ques'])
df['ans'] = get_lemmatized_text(df['ans'])

# Convert numbers to letters

def convert_num_to_words(utterance):
      utterance = ' '.join([num2words(i) if i.isdigit() else i for i in utterance.split()])
      return utterance


df["category"] = df["category"].apply(lambda text: convert_num_to_words(str(text)))
df["ans"] = df["ans"].apply(lambda text: convert_num_to_words(str(text)))
df["ques"] = df["ques"].apply(lambda text: convert_num_to_words(str(text)))

df = df.sample(frac=1).reset_index(drop=True) # reset index
df

,shownum,airdate,round,category,value,ques,ans
0,4929,2006-02-02,Double Jeopardy!,napoleon time,$1200,"on sept fourteen one thousand, eight hundred a...",moscow
1,4721,2005-02-28,Jeopardy!,the jimmi carter experi,$600,cool under fire dure a seven year navi career ...,a nuclear submarin
2,6000,2010-10-15,Jeopardy!,four letter word,$800,french for flea thi four letter color is a dar...,puce
3,5084,2006-10-19,Double Jeopardy!,the music dr is in,$2000,despit it name thi synth pop english band who ...,the thompson twin
4,4328,2003-05-28,Jeopardy!,fix the malaprop,$600,certain area of the desert are cultiv by irrit,irrig
...,...,...,...,...,...,...,...
99995,3577,2000-03-07,Double Jeopardy!,who want to be a millionair ?,$1000,son of the owner of harrod he die in a tragic ...,dodi al fay
99996,1175,1989-10-13,Jeopardy!,singer,$500,when thi duo record hey schoolgirl in one thou...,simon garfunkel
99997,61,1984-12-03,Jeopardy!,four letter word,$500,bacal 's last word on how to whistl,blow
99998,4174,2002-10-24,Jeopardy!,odd end,$200,cheryl of the clue crew stand in front of a ma...,chimney sweep


In [410]:
del df['shownum']  # deleting shownum and airdate rows as they are not important features
del df['airdate']


In [411]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,round,category,value,ques,ans
0,Jeopardy!,beatl wannab ?,$800,imagin thoma lennon 's success with thi comedi...,reno nine hundred and eleven
1,Double Jeopardy!,biblic citi,$1600,on the road to thi citi paul wa struck blind f...,damascu
2,Double Jeopardy!,day s of wrath,$200,start pray now it 's the day at the end of the...,judgment day
3,Jeopardy!,presid tyler,$1000,"with one thousand, eight hundred and forty-two...",main
4,Jeopardy!,avoid a disastr trip to europ,$200,tri not to leav a href http www j archiv com m...,the parthenon


# Data Filtering

In [412]:
df = df[df['round'] == 'Jeopardy!']  # select only Jeopardy round data
#print(df.shape)
df = df[df.ans.str.isalpha() == True]  # selecting only those rows in which ans is a string

df.shape


(20707, 5)

In [413]:
# Can also delete round too as it is constant now because we are only considering Jeopardy round
del df['round']

In [414]:
df = df[~df.ques.str.contains("<a href=")]
df.shape

(20707, 4)

In [415]:
# Filter questions where len of ques is equal or more than 5
df = df[df['ques'].str.split().str.len() >= 5]
df.shape

(19283, 4)

In [416]:
df = df.sample(frac=1).reset_index(drop=True)
df

,category,value,ques,ans
0,eye on indonesia,$300,amid a muslim nation thi island next to java m...,bali
1,intern cuisin,$400,thi german dish whose name mean hare pepper is...,hassenpfeff
2,choos your weapon,$1000,put descart befor the hors is an exampl of one...,pun
3,celebrities' middl name,$600,author hawthorn would approv of thi middl name...,nathaniel
4,ow,$1000,it 's a comparit short barrel cannon use to re...,howitz
...,...,...,...,...
19278,whoop,$400,thi aegean countri increas it gdp by includ th...,greec
19279,collect the whole seri,$600,a horseman of the apocalyps trade card set wou...,death
19280,how inspir,$1000,thi candid author help popular the say the per...,voltair
19281,it' element,$600,in the electron industri thi element symbol si...,silicon


In [417]:
# Final Data Prepraration
# Class Balanced Data with 50% samples with $200 and vice-versa
df_200 = df[df['value']=='$200'].sample(2000, random_state=670)
df_1000 = df[df['value']=='$1000'].sample(2000, random_state=670)
df = pd.concat([df_200,df_1000])
df = df.sample(frac=1).reset_index(drop=True)
df

,category,value,ques,ans
0,offici state seal,$200,it seal show a map of the state with georgia o...,alabama
1,up on mount rushmor,$1000,appoint oliv ellsworth chief justic,washington
2,let the game begin,$1000,a href http www j archiv com medium two thousa...,grace
3,thoma pain,$200,with letter of introduct from ben franklin tho...,philadelphia
4,the new testament,$200,chapter six of thi new testament book describ ...,revel
...,...,...,...,...
3995,literari movement,$200,thi movement that includ shelley byron wa base...,romantic
3996,cocoa channel,$200,there are fruiti cocoa varieti of thi cereal p...,pebbl
3997,travel tourism,$1000,about half of the visitor to thi former portug...,macao
3998,desert,$200,roald dahl 's augustu gloop get hi just de s e...,chocol


In [418]:
# Saving the modified data, will be used later 
df.to_csv('/home/mayank/Desktop/mod_data.csv')

In [427]:
# Loading the new data
df = pd.read_csv("/home/mayank/Desktop/mod_data.csv")
df

,Unnamed: 0,category,value,ques,ans
0,0,offici state seal,$200,it seal show a map of the state with georgia o...,alabama
1,1,up on mount rushmor,$1000,appoint oliv ellsworth chief justic,washington
2,2,let the game begin,$1000,a href http www j archiv com medium two thousa...,grace
3,3,thoma pain,$200,with letter of introduct from ben franklin tho...,philadelphia
4,4,the new testament,$200,chapter six of thi new testament book describ ...,revel
...,...,...,...,...,...
3995,3995,literari movement,$200,thi movement that includ shelley byron wa base...,romantic
3996,3996,cocoa channel,$200,there are fruiti cocoa varieti of thi cereal p...,pebbl
3997,3997,travel tourism,$1000,about half of the visitor to thi former portug...,macao
3998,3998,desert,$200,roald dahl 's augustu gloop get hi just de s e...,chocol


# Finetuning Fastext model on pretrained wikipedia news model

In [428]:
# Generate fasttext model for "ques","ans","category" col separately

df_new = df
del df_new["ans"]
del df_new["ques"]
df_new




,Unnamed: 0,category,value
0,0,offici state seal,$200
1,1,up on mount rushmor,$1000
2,2,let the game begin,$1000
3,3,thoma pain,$200
4,4,the new testament,$200
...,...,...,...
3995,3995,literari movement,$200
3996,3996,cocoa channel,$200
3997,3997,travel tourism,$1000
3998,3998,desert,$200


In [425]:
# Fast Text 
with open("/home/mayank/Desktop/ques.txt", "w") as train_file_handler:
    for X_train_entry, y_train_entry in zip(df_new['category'],df_new['value']):
        line_to_write = "__label__" + str(y_train_entry) + "\t" + str(X_train_entry) + "\n"
        try:
            train_file_handler.write(line_to_write)
        except:
            print(line_to_write)
            break
        



In [429]:
# These fastext model will be used later
import re
import fasttext
import numpy as np
import pandas as pd
from fasttext import load_model

hyper_params = { 
    "lr": 0.001,         # Learning rate
    "epoch": 25,       # Number of training epochs to train for
    "wordNgrams": 4,    # Number of word n-grams to consider during training
    "dim": 300,         # Size of word vectors
    "ws": 5,        # Size of the context window for CBOW or skip-gram
    'lrUpdateRate': 15000,
    'bucket' : 100000,
    "loss" : 'ns',
    "maxn" : 5,
    "minn" : 2,
    "minCount":1
    }
print("Training Started")
model = fasttext.train_supervised(input='/home/mayank/Desktop/ques.txt',**hyper_params,verbose = 10,pretrainedVectors='/home/mayank/Downloads/pmeans/wiki-news-300d-1M.vec')
print("Training Ended")



print("Saving the model...........")
model.save_model('/home/mayank/Desktop/cat_model.bin')
print("Model saved successfully")

Training Started
Training Ended
/n/nSaving the model...........
/n/n Model saved successfully
